In [43]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [44]:
df = pd.read_csv('Churn_Modelling.csv')

In [45]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [46]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [47]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [48]:
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [49]:
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [50]:
#Encoding
encode_gender = LabelEncoder()
df['Gender']=encode_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [51]:
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder(sparse=False)

In [53]:
geo_encoded = ohe_geo.fit_transform(df[['Geography']])
geo_encoded


c:\Users\wajid\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [57]:
type(geo_encoded)

numpy.ndarray

In [54]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [58]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [59]:
#combine one hot encode columns to original data
df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [60]:
# save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(encode_gender, file)

with open('onehot_encoder_gender.pkl', 'wb') as file:
    pickle.dump(ohe_geo, file)  


In [61]:
from sklearn.model_selection import train_test_split
x = df.drop('Exited', axis=1)
y = df['Exited']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [62]:
x_test_scaled

array([[-0.58163494,  0.91186722, -0.65514605, ..., -0.99946681,
         1.73082   , -0.57735027],
       [-0.30112032,  0.91186722,  0.3926101 , ...,  1.00053348,
        -0.57776083, -0.57735027],
       [-0.52968779, -1.09665089,  0.48786066, ..., -0.99946681,
        -0.57776083,  1.73205081],
       ...,
       [ 1.20534708,  0.91186722, -1.32189996, ...,  1.00053348,
        -0.57776083, -0.57735027],
       [ 0.39497151, -1.09665089, -0.27414381, ...,  1.00053348,
        -0.57776083, -0.57735027],
       [-0.4985195 , -1.09665089, -1.32189996, ..., -0.99946681,
         1.73082   , -0.57735027]])

In [63]:
x_test_scaled

array([[-0.58163494,  0.91186722, -0.65514605, ..., -0.99946681,
         1.73082   , -0.57735027],
       [-0.30112032,  0.91186722,  0.3926101 , ...,  1.00053348,
        -0.57776083, -0.57735027],
       [-0.52968779, -1.09665089,  0.48786066, ..., -0.99946681,
        -0.57776083,  1.73205081],
       ...,
       [ 1.20534708,  0.91186722, -1.32189996, ...,  1.00053348,
        -0.57776083, -0.57735027],
       [ 0.39497151, -1.09665089, -0.27414381, ...,  1.00053348,
        -0.57776083, -0.57735027],
       [-0.4985195 , -1.09665089, -1.32189996, ..., -0.99946681,
         1.73082   , -0.57735027]])

In [64]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

# ANN Implementation

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [67]:
x_train_scaled.shape[1]

12

In [69]:
# Build ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train_scaled.shape[1], )), #Hidden Layer 1 connected to input layer
    Dense(32, activation='relu'), #Hl2
    Dense(1, activation='sigmoid') #o/p layer
])

In [70]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

'''optimize = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()
loss'''

<LossFunctionWrapper(<function binary_crossentropy at 0x000002580627D3A0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [74]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
# Setup the TensorBoard
log_dir = 'log/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir, histogram_freq=1)

In [80]:
#Setup early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [81]:
# Training the model
history = model.fit(
    x_train_scaled, y_train, validation_data = (x_test_scaled, y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8677 - loss: 0.3169 - val_accuracy: 0.8604 - val_loss: 0.3396
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8639 - loss: 0.3165 - val_accuracy: 0.8632 - val_loss: 0.3419
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8700 - loss: 0.3144 - val_accuracy: 0.8644 - val_loss: 0.3395
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8683 - loss: 0.3131 - val_accuracy: 0.8620 - val_loss: 0.3368
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8689 - loss: 0.3127 - val_accuracy: 0.8608 - val_loss: 0.3398
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8680 - loss: 0.3104 - val_accuracy: 0.8596 - val_loss: 0.3425
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8696 - loss: 0.3093 - val_accuracy: 0.8640 - val_loss: 0.3398
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8716 - loss: 0.3091 - val_accu

In [82]:
model.save('model.h5')

In [83]:
# Load TensorBoard extension
%load_ext tensorboard

In [91]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6007 (pid 9324), started 0:07:17 ago. (Use '!kill 9324' to kill it.)